In [36]:
import tensorflow as tf
import numpy as np 
import pandas as pd

In [43]:
df = pd.read_csv('download/sunspots.csv', index_col=0)
time_step = list(range(len(df)))
sunspots = df['Monthly Mean Total Sunspot Number'].values

series = np.array(sunspots)
time = np.array(time_step)

In [30]:
window_size = 5
dataset = tf.data.Dataset.range(10)
dataset = (dataset
    # Create windowed dataset
    .window(window_size, shift=1, drop_remainder=True)
    # Maps function and flattens result
    .flat_map(lambda window: window.batch(window_size))
    # Split into x, y
    .map(lambda window: (window[:-1], window[-1:]))
    # Shuffle 10 bec we have 10 items - not necessary for time series
    .shuffle(buffer_size=10)
    # Results in 3x batches of 2 items each
    .batch(2)
    .prefetch(tf.data.AUTOTUNE)
)
for x, y in dataset:
    print('x = ', x.numpy())
    print('y = ', y.numpy())

x =  [[3 4 5 6]
 [2 3 4 5]]
y =  [[7]
 [6]]
x =  [[5 6 7 8]
 [4 5 6 7]]
y =  [[9]
 [8]]
x =  [[0 1 2 3]
 [1 2 3 4]]
y =  [[4]
 [5]]


In [31]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = (tf.data.Dataset.from_tensor_slices(series)
        .window(window_size+1, shift=1, drop_remainder=True)
        .flat_map(lambda window: window.batch(window_size+1))
        # Does it matter if we shuffle before or after the map
        .shuffle(shuffle_buffer)
        .map(lambda window: (window[:-1], window[-1:]))
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataset

In [ ]:
split_time = 3000
time_train = time[:split_time]
X_train = series[:split_time]

time_val = time[split_time:]
X_val = time[split_time:]


window_size = 30
batch_size = 32
shuffle_buffer_size = 1000

dataset = windowed_dataset(series, window_size, batch_size, shuffle_buffer_size)
# Simple Linear Regression
L0 = tf.keras.layers.Dense(1, input_shape=window_size)
model = tf.keras.models.Sequential([L0])

model.compile(
    loss='mse',
    optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9)
)
model.fit(dataset, epochs=100)

In [ ]:
# 20 weights (since input size is 20 lag vars)
# and one b intercept value
print(f'Layer weights {L0.get_weights()}')

In [ ]:
print(series[1:21])
model.predict(series[1:21][np.newaxis])

In [ ]:
# How to forecast a time series
forecast = []
for time in range(len(series) - window_size):
    pred = model.predict(series[time:time +  window_size][np.newaxis])
    forecast.append(pred)
# Split out for plotting
forecast = [split_time - window_size:]
results = np.array(forecast)[:, 0, 0]

In [ ]:
tf.keras.metrics.mean_absolute_error(X_val, results).numpy()

In [ ]:
# Learning rate scheduling and picking optimal LR
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch/20)
)
# Plot
lrs = 1e-8 * (10 ** (np.arange(100)/20))
# Just plot learning rates against the loss! Easy.
plt.semilogx(lrs, history.history['loss'])
plt.axis([1e-8, 1e-4, 0, 300])

Simple RNNs and LSTMs will give better performance. Swap out `SimpleRNN` below for `LSTM` and/or `Bidirectional(LSTM)` for better performance. 

In [ ]:
model = tf.keras.models.Sequential([
    # Expand dims so input is 3D (and no need to modify input pipeline)
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                            input_shape=[None]), # model takes seqs of any length
    tf.keras.layers.SimpleRNN(20, return_sequences=True),
    tf.keras.layers.SimpleRNN(20),
    tf.keras.layers.Dense(1),
    # Data is in the 40-70 range but tanh outputs in [-1, 1]
    # so scaling preds to the actual range can help
    tf.keras.layers.Lambda(lambda x: x * 100.0)
])

Using `Conv1D` + `LSTM` layers will almost certainly result in best performance.

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Lambda, Bidirectional

In [ ]:
conv_params = dict(
    filters=32, 
    kernel_size=5,
    strides=1, 
    padding='causal', # appropriate padding for temporal data
    activation='relu',
    input_shape=[None, 1],
)
# Note: for this to work, need to add the following to 
# 1st line of windowed_dataset func:
# series = tf.expand_dims(series, axis=-1)
model = Sequential([
    Conv1D(**conv_params),
    Bidirectional(LSTM(32, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(1),
    Lambda(lambda x: x * 200)
])